In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import data
df = pd.read_csv('../data/AviationData.csv', encoding='latin-1')
state_codes = pd.read_csv('../data/USState_Codes.csv')

### Initial Data Cleaning

In [ ]:
# Change the date column to a datetime object
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
# Creating a column for the year and insert next to Event.Date
df['Year'] = df['Event.Date'].dt.year
df.insert(4, 'Year', df.pop('Year'))

# Make everything in the df lowercase so we can get more accurate counts
df = df.applymap(lambda x: x.lower() if type(x) == str else x)
# Renaming columns in titlecase (aesthetic purposes only)
df = df.rename(columns={'Event.Id':'Event.ID','Aircraft.damage':'Aircraft.Damage','Purpose.of.flight':'Purpose.of.Flight','Air.carrier':'Air.Carrier','Broad.phase.of.flight':'Broad.Phase.of.Flight'})

df.info()

### Geographic Filtering

In [ ]:
# Filter to include only US events and remove 'Country' column
df = df[df['Country'] == 'united states']
df = df.drop(columns=['Country'])

In [ ]:
# Create new column 'State.Code' pulling info from 'Location'
df['State.Code'] = df['Location'].str.split(',').str[-1].str.strip().str.upper()
df = df.dropna(subset=['State.Code'])

# Create new column 'State.Names' merging info from state_codes df
df = pd.merge(df, state_codes, how='left', left_on='State.Code', right_on='Abbreviation')
df = df.drop(['Abbreviation'], 1)
df = df.rename(columns={'US_State':'State.Name'})
df = df.dropna(subset=['State.Name'])

# Move both columns next to 'Location'
df.insert(6, 'State.Code', df.pop('State.Code'))
df.insert(7, 'State.Name', df.pop('State.Name'))

In [ ]:
# Map states to regions based off data from http://nationalgeographic.org/maps/united-states-regions/
# DC is categorized as a state
# Atlantic and Pacific categorized as 'Ocean'
# Guam, PR, VI, Gulf of Mexico categorized as 'Other Territory'

states_to_regions = {
    'Washington': 'West', 'Oregon': 'West', 'California': 'West', 'Nevada': 'West',
    'Idaho': 'West', 'Montana': 'West', 'Wyoming': 'West', 'Utah': 'West',
    'Colorado': 'West', 'Alaska': 'West', 'Hawaii': 'West', 'Maine': 'Northeast',
    'Vermont': 'Northeast', 'New York': 'Northeast', 'New Hampshire': 'Northeast',
    'Massachusetts': 'Northeast', 'Rhode Island': 'Northeast', 'Connecticut': 'Northeast',
    'New Jersey': 'Northeast', 'Pennsylvania': 'Northeast', 'North Dakota': 'Midwest',
    'South Dakota': 'Midwest', 'Nebraska': 'Midwest', 'Kansas': 'Midwest',
    'Minnesota': 'Midwest', 'Iowa': 'Midwest', 'Missouri': 'Midwest', 'Wisconsin': 'Midwest',
    'Illinois': 'Midwest', 'Michigan': 'Midwest', 'Indiana': 'Midwest', 'Ohio': 'Midwest',
    'West Virginia': 'South', 'Maryland': 'South', 'Virginia': 'South', 'Kentucky': 'South', 
    'Tennessee': 'South', 'North Carolina': 'South', 'Mississippi': 'South', 'Arkansas': 'South', 
    'Louisiana': 'South', 'Alabama': 'South', 'Georgia': 'South', 'South Carolina': 'South', 
    'Florida': 'South', 'Delaware': 'South', 'Arizona': 'Southwest', 'New Mexico': 'Southwest', 
    'Oklahoma': 'Southwest', 'Texas': 'Southwest', 'Washington_DC': 'South', 
    'Guam': 'Other Territory', 'Puerto Rico': 'Other Territory', 'Virgin Islands': 'Other Territory',
     'Gulf of mexico': 'Other Territory', 'Atlantic ocean': 'Ocean', 'Pacific ocean': 'Ocean' }

df['Region'] = df['State.Name'].map(states_to_regions)
df.insert(8, 'Region', df.pop('Region'))
df['Region'].value_counts()

### Aircraft Type

In [ ]:
# Filter out 'Amateur.Built' aircraft and remove column
df = df[df['Amateur.Built'] == 'no']
df = df.drop(columns=['Amateur.Built'])

In [ ]:
# One interesting thing to note here is the difference in the number of events per year
# Here is the original data *before* we filter out non-airplane events

df['Year'].value_counts().sort_index().plot(kind='bar', figsize=(15, 5))

In [ ]:
# We can see that the airplane-related incidents and the non-airplane-related incidents are mirror images of each other

df_notplanes = df[df['Aircraft.Category'] != 'airplane']
df_notplanes['Year'].value_counts().sort_index().plot(kind='bar', figsize=(15, 5))

In [ ]:
# Filter to include only 'airplane' events and remove 'Aircraft.Category' column
df = df[df['Aircraft.Category'] == 'airplane']
df = df.drop(columns=['Aircraft.Category'])

# Regraph after filtering
df['Year'].value_counts().sort_index().plot(kind='bar', figsize=(15, 5))

### Time Frame

In [ ]:
# Given the above data and knowing that technology has advanced exponentially in the past two decades
# we decided to conduct our analysis on just the past 15 years of data- regulatary requirements may also have been updated 
# df = df[df['Year'] > 2007]

### FAR Codes & Purpose of Flight

In [ ]:
df['FAR.Description'].value_counts()

In [ ]:
# Cleaning up the FAR.Description column using .replace() 
# See here for more info on FAR: 
# https://pilotinstitute.com/part-91-vs-121-vs-135/

# Part 91 - general aviation
df['FAR.Description'] = df['FAR.Description'].replace(['091','part 91: general aviation','091k','part91f','part 91 subpart f','part 91f: special flt ops.','part 91 subpart k: fractional'], 'part 91')

# Part 121 - scheduled commercial (airlines, cargo)
df['FAR.Description'] = df['FAR.Description'].replace(['121','part 121: air carrier'], 'part 121')

# Part 135 - non-scheduled commercial (private jet, air taxi)
df['FAR.Description'] = df['FAR.Description'].replace(['135','part 135: air taxi & commuter'], 'part 135')

# Part 137 - agricultural 
df['FAR.Description'] = df['FAR.Description'].replace(['137','part 137: agricultural'], 'part 137')

# Part 129 - foreign air carriers
df['FAR.Description'] = df['FAR.Description'].replace(['129','part 129: foreign'], 'part 129')

# NUSC - non-US commercial
# NUSN - non-US non-commercial
df['FAR.Description'] = df['FAR.Description'].replace(['nusc','nusn'], 'non-US')

# Part 125 - 20+ passengers
df['FAR.Description'] = df['FAR.Description'].replace(['125','part 125: 20+ pax,6000+ lbs'], 'part 125')

# Part 133 - helicopters
df['FAR.Description'] = df['FAR.Description'].replace(['133','part 133: rotorcraft external load','part 133: rotorcraft ext. load'], 'part 133')

# Public
df['FAR.Description'] = df['FAR.Description'].replace(['pubu','public use','public aircraft'], 'public')

# Unknown
df['FAR.Description'] = df['FAR.Description'].replace(['unk'], 'unknown')

# Armed Forces
df['FAR.Description'] = df['FAR.Description'].replace(['armf'], 'armed forces')

df['FAR.Description'].value_counts()

In [ ]:
df['Purpose.of.Flight'].value_counts()

In [ ]:
# New column 'Purpose.Binned' to group 'Purpose.of.Flight' into more general categories

# New category 'public - govt' to include 'public aircraft - federal', 'public aircraft - state', 'public aircraft - local', 'public aircraft - u.s. govt.', 'pubs', 'firefighting' 
df['Purpose.Binned'] = df['Purpose.of.Flight'].replace(['public aircraft - federal', 'public aircraft - state', 'pubs', 'public aircraft - local', 'publ', 'public aircraft - u.s. govt.','firefighting'], 'public - govt')

# New category 'public transport' to include 'public aircraft',
df['Purpose.Binned'] = df['Purpose.Binned'].replace(['public aircraft'], 'public transport')

# New category 'business' to include 'business', 'other work use', 'banner tow', 'air drop'
df['Purpose.Binned'] = df['Purpose.Binned'].replace(['business', 'other work use', 'banner tow', 'air drop'], 'business')

# New category 'recreational' to include 'skydiving', 'glider tow'
df['Purpose.Binned'] = df['Purpose.Binned'].replace(['skydiving', 'glider tow'], 'recreational')

# New category 'private transport' to include 'executive/corporate', 'ferry'
df['Purpose.Binned'] = df['Purpose.Binned'].replace(['executive/corporate', 'ferry'], 'private transport')

# New category 'air show' to include 'air race show', 'air race/show', 'asho'
df['Purpose.Binned'] = df['Purpose.Binned'].replace(['air race show', 'air race/show', 'asho'], 'air show')

df.insert(23, 'Purpose.Binned', df.pop('Purpose.Binned'))
df['Purpose.Binned'].value_counts()

### Filtering the 'Make' series


In [ ]:
# Checking out the raw 'Make' values
df['Make'].value_counts()

In [ ]:
# Make a list of all the entries that have "cessna" in them and return their strings in a list
cessna_list = df[df['Make'].str.contains('cessna')]['Make'].unique().tolist()

#Do the same with the other big makes
grumman_list = df[df['Make'].str.contains('grumman')]['Make'].unique().tolist()
dehavilland_list = df[df['Make'].str.contains('havilland')]['Make'].unique().tolist()
cirrus_list = df[df['Make'].str.contains('cirrus')]['Make'].unique().tolist()
piper_list = df[df['Make'].str.contains('piper')]['Make'].unique().tolist()
boeing_list = df[df['Make'].str.contains('boeing')]['Make'].unique().tolist()
mooney_list = df[df['Make'].str.contains('mooney')]['Make'].unique().tolist()
airtractor_list = df[df['Make'].str.contains('air tractor')]['Make'].unique().tolist()

print(cessna_list)
print(grumman_list)
print(dehavilland_list)
print(cirrus_list)
print(piper_list)
print(boeing_list)
print(mooney_list)
print(airtractor_list)

In [ ]:
df['Make'] = df['Make'].replace(cessna_list, 'cessna')
df['Make'] = df['Make'].replace(grumman_list, 'grumman')
df['Make'] = df['Make'].replace(dehavilland_list, 'dehavilland')
df['Make'] = df['Make'].replace(cirrus_list, 'cirrus')
df['Make'] = df['Make'].replace(piper_list, 'piper')
df['Make'] = df['Make'].replace(mooney_list, 'mooney')
df['Make'] = df['Make'].replace(airtractor_list, 'airtractor')

# And we can now see the totals, with slightly different numbers than before.
df['Make'].value_counts()

In [ ]:
# Making a new df that only accepts the top 25 makes:
top_makes = df['Make'].value_counts().head(25).index.tolist()
top_makes_df = df[df['Make'].isin(top_makes)].copy()
top_makes_df['Make'].value_counts()

### Airports

In [ ]:
# Clean airport names

df['Airport.Name'] = df['Airport.Name'].replace(['private airstrip', 'private strip', 'pvt'], 'private')
df['Airport.Name'].value_counts()

### Weather

In [ ]:
# Clean 'Weather.Condition'

df['Weather.Condition'] = df['Weather.Condition'].str.upper()
df['Weather.Condition'].value_counts()

# VMC - Visual Meteorological Conditions - generally clear and good visibility; pilots can navigate and operate aircraft by visual reference to the ground
# IMC - Instrument Meteorological Conditions - reduced visibility due to factors like fog, rain, or low clouds; pilots may need to rely on instruments for navigation and control
# UNK - Unknown

### Injury Severity

In [ ]:
# Looking at the 'Injury.Severity' column
# We see that it tells us how many people died in a given incident
df['Injury.Severity'].value_counts().head(20)

In [ ]:
# Split number from Fatal(#) to count number of fatalities
# Create a new column tracking the number of these fatalities
df['Num.Fatalities'] = df['Injury.Severity'].str.split('(').str[-1]
df['Num.Fatalities'] = df['Num.Fatalities'].str.split(')').str[0]
df['Num.Fatalities'] = pd.to_numeric(df['Num.Fatalities'], errors='coerce').fillna(0).astype(int)
df.insert(14, 'Num.Fatalities', df.pop('Num.Fatalities'))
df['Num.Fatalities'].value_counts()

In [ ]:
# Update labeling in 'Injury.Severity' column, since we now have a different column that tracks total number of fatalities

df['Injury.Severity'] = df['Injury.Severity'].str.split('(').str[0]
df['Injury.Severity'].value_counts()

In [ ]:
# Move other injury columns next to Injury.Severity
df.insert(15, 'Total.Fatal.Injuries', df.pop('Total.Fatal.Injuries'))
df.insert(16, 'Total.Serious.Injuries', df.pop('Total.Serious.Injuries'))
df.insert(17, 'Total.Minor.Injuries', df.pop('Total.Minor.Injuries'))
df.insert(18, 'Total.Uninjured', df.pop('Total.Uninjured'))

### Damage Severity Score

Making a 'DS.Score' column that takes the 'Aircraft.damage' column and reassigns each category a number:

    - 'destroyed' -> 2
    - 'substantial' -> 1
    - 'minor' -> 0
    
For the time being, 'unknown' is left as such.

In [ ]:
# Make a DS.Score based on the 'Aircraft.Damage' column

df['DS.Score'] = df['Aircraft.Damage'].replace({
    'destroyed': 2,
    'substantial': 1,
    'minor': 0,
})
df['DS.Score'].value_counts()

df.insert(20, 'DS.Score', df.pop('DS.Score'))

# Save cleaned .csv

In [ ]:
df.info()

In [ ]:
# Save cleaned df to a new csv file
# df.to_csv('../data/AviationData_cleaned.csv', index=False)